In [51]:
import pandas as pd
import numpy as np
from datetime import timedelta
import random

# Параметры
n_contracts = 1000       # сколько уникальных контрактов сгенерировать
start_date = pd.to_datetime("2023-01-01")
end_date = pd.to_datetime("2025-05-01")

statuses = [
    "Продан (закрыт)", "Отказ заемщика от займа", "Клиент отказался от кредита",
    "Отказ Банком", "Передан на совершение испол.надписи", "Актуален",
    "Передан в суд", "Списан долг", "Договор завершен",
    "Отсутствует на источнике", "Прощен", "Продан", "Погашен",
    "Отказ от кредита", "Списан", "Погашен по решению суда",
    "Удалён физически", "Реф по ГП", "Погашен ГП", "Завершение",
    "Удален", "Договор введен", "Отказ Заемщиком", "Списаны проценты",
    "Отказ заемщиком", "Зарегистрирован", "Графики рассчитаны", "Введен"
]
currencies = ["KZT", "USD", "EUR"]
sign_restruct_options = ["Yes", "No"]
branch_names = [f"Филиал {i}" for i in range(1, 11)]
loan_purposes = [
    ("LP01", "Покупка жилья"), ("LP02", "Ремонт"), ("LP03", "Покупка авто"),
    ("LP04", "Потребительский"), ("LP05", "Туризм")
]
collector_companies = ["CompanyA", "CompanyB", "CompanyC"]

# Уникальные product_code (пример, замените на свой полный список)
ALL_CODES = [f"10.FORT_IND_CR_{i:04d}" for i in range(1, 50)]
product_names = [f"Product {i}" for i in range(len(ALL_CODES))]

# --- Новый блок: генерируем пул BIN/IIN с переиспользованием ---
n_clients = int(n_contracts * 0.7)             # 70% контракты будут на уникальных клиентов
client_bins = [
    "".join(str(random.randint(0, 9)) for _ in range(12))
    for _ in range(n_clients)
]
# Для контрактов случайно выбираем BIN/IIN из этого пула (появятся дубликаты)
assigned_bins = random.choices(client_bins, k=n_contracts)
# ------------------------------------------------------------

# 1) Генерируем статические данные контрактов
contracts = []
for i in range(n_contracts):
    open_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days - 30))
    if random.random() < 0.7:
        close_date = open_date + timedelta(days=random.randint(30, (end_date - open_date).days))
    else:
        close_date = None
    if close_date and random.random() < 0.5:
        liquidate_date = open_date + timedelta(days=random.randint(1, (close_date - open_date).days))
    else:
        liquidate_date = None

    code = random.choice(ALL_CODES)
    rate = round(random.uniform(5.0, 25.0), 2)
    eff_rate = round(rate + random.uniform(0, 5.0), 2)

    contracts.append({
        "contract_number": f"CN{random.randint(100000, 999999)}",
        "bin_iin": assigned_bins[i],             # используем повторяющиеся BIN/IIN
        "open_date": open_date,
        "close_date": close_date,
        "liquidate_date": liquidate_date,
        "currency": random.choice(currencies),
        "sign_restructing": random.choice(sign_restruct_options),
        "branch_code": str(random.randint(0, 10)),
        "branch_name": random.choice(branch_names),
        "loan_purpose_code": random.choice([lp[0] for lp in loan_purposes]),
        "loan_purpose_name": random.choice([lp[1] for lp in loan_purposes]),
        "collector_company": random.choice(collector_companies),
        "product_code": code,
        "product_name": product_names[ALL_CODES.index(code) % len(product_names)],
        "contract_amount": round(random.uniform(100_000, 5_000_000), 2),
        "interest_rate": rate,
        "eff_interest_rate": eff_rate,
    })

# 2) Для каждого контракта создаём строки по каждому дню от открытия до end_date
rows = []
for c in contracts:
    current = c["open_date"]
    while current <= end_date:
        status = random.choice(statuses)
        if c["close_date"] and current >= c["close_date"]:
            balance_main = 0.0
        else:
            balance_main = round(random.uniform(0, c["contract_amount"]), 2)
        balance_int = round(random.uniform(0, c["contract_amount"] * 0.1), 2)

        rows.append({
            "act_date": current.date(),
            **{k: v for k, v in c.items() if k not in ["close_date","liquidate_date"]},
            "agreem_state": status,
            "close_date": c["close_date"].date() if c["close_date"] else None,
            "liquidate_date": c["liquidate_date"].date() if c["liquidate_date"] else None,
            "delay_count_main": random.choice([None] + list(range(0, 120))),
            "delay_count_percent": random.choice([None] + list(range(0, 120))),
            "balance_main_dbt_amt_in_lcl_ccy": balance_main,
            "balance_int_amt_in_lcl_ccy": balance_int,
            "next_payment_date": (current + timedelta(days=random.randint(30,90))).date(),
            "z_good": random.choice(["Good_bank","Heritage","Stressful"])
        })
        current += timedelta(days=1)

# 3) Собираем DataFrame
df = pd.DataFrame(rows)

# Проверка
print(df.shape)
print(df["bin_iin"].nunique(), "уникальных BIN/IIN на", n_contracts, "контрактов")
print(df.head())


(443457, 25)
513 уникальных BIN/IIN на 1000 контрактов
     act_date contract_number       bin_iin  open_date currency  \
0  2023-06-23        CN489252  824652509913 2023-06-23      USD   
1  2023-06-24        CN489252  824652509913 2023-06-23      USD   
2  2023-06-25        CN489252  824652509913 2023-06-23      USD   
3  2023-06-26        CN489252  824652509913 2023-06-23      USD   
4  2023-06-27        CN489252  824652509913 2023-06-23      USD   

  sign_restructing branch_code branch_name loan_purpose_code  \
0              Yes           7    Филиал 4              LP02   
1              Yes           7    Филиал 4              LP02   
2              Yes           7    Филиал 4              LP02   
3              Yes           7    Филиал 4              LP02   
4              Yes           7    Филиал 4              LP02   

  loan_purpose_name collector_company         product_code product_name  \
0            Туризм          CompanyB  10.FORT_IND_CR_0011   Product 10   
1      

In [52]:
df['delay_count_main'] = pd.to_numeric(df['delay_count_main'], errors='coerce').astype('Int64')
df['delay_count_percent'] = pd.to_numeric(df['delay_count_percent'], errors='coerce').astype('Int64')


In [53]:
pd.set_option('display.max_columns', None)


In [54]:
df['bin_iin'].nunique()

513

In [55]:
df['contract_number'].nunique()

999

In [56]:
print(df.columns.tolist())


['act_date', 'contract_number', 'bin_iin', 'open_date', 'currency', 'sign_restructing', 'branch_code', 'branch_name', 'loan_purpose_code', 'loan_purpose_name', 'collector_company', 'product_code', 'product_name', 'contract_amount', 'interest_rate', 'eff_interest_rate', 'agreem_state', 'close_date', 'liquidate_date', 'delay_count_main', 'delay_count_percent', 'balance_main_dbt_amt_in_lcl_ccy', 'balance_int_amt_in_lcl_ccy', 'next_payment_date', 'z_good']


In [58]:
import pandas as pd
import psycopg2
import io
from io import StringIO

conn = psycopg2.connect(
    host="127.0.0.1",
    port="5432",
    database="postgres",
    user="postgres",
    password="admin"
)

cur = conn.cursor()

# --- Создание таблицы ---
create_table_sql = """
CREATE TABLE IF NOT EXISTS credit_portfolio (
    act_date DATE,
    contract_number VARCHAR,
    agreem_state VARCHAR,
    contract_amount NUMERIC,
    open_date DATE,
    close_date DATE,
    liquidate_date DATE,
    currency VARCHAR,
    sign_restructing VARCHAR,
    branch_code VARCHAR,
    branch_name VARCHAR,
    loan_purpose_code VARCHAR,
    loan_purpose_name VARCHAR,
    collector_company VARCHAR,
    product_code VARCHAR,
    product_name VARCHAR,
    bin_iin VARCHAR,
    interest_rate NUMERIC,
    eff_interest_rate NUMERIC,
    delay_count_main INTEGER,
    delay_count_percent INTEGER,
    balance_main_dbt_amt_in_lcl_ccy NUMERIC,
    balance_int_amt_in_lcl_ccy NUMERIC,
    next_payment_date DATE,
    z_good VARCHAR
);
"""
cur.execute(create_table_sql)
conn.commit()

# --- Загрузка данных в таблицу ---
# Преобразуем DataFrame в CSV-строку






output = StringIO()
# Правильный порядок колонок
columns = [
    "act_date", "contract_number", "agreem_state", "contract_amount",
    "open_date", "close_date", "liquidate_date", "currency",
    "sign_restructing", "branch_code", "branch_name", "loan_purpose_code",
    "loan_purpose_name", "collector_company", "product_code", "product_name",
    "bin_iin", "interest_rate", "eff_interest_rate", "delay_count_main",
    "delay_count_percent", "balance_main_dbt_amt_in_lcl_ccy",
    "balance_int_amt_in_lcl_ccy", "next_payment_date", "z_good"
]

# Только эти колонки
df_to_load = df[columns]

# Пишем как TSV без заголовка и index
output = io.StringIO()
df_to_load.to_csv(output, sep='\t', index=False, header=False, na_rep='')

output.seek(0)
cur.copy_from(output, 'credit_portfolio', sep='\t', null='')

# Завершаем транзакцию
conn.commit()

# Закрываем соединение
cur.close()
conn.close()

print("✅ Таблица создана и данные загружены.")

✅ Таблица создана и данные загружены.


In [67]:
credit_df=pd.read_excel(r"C:\Users\admin\Desktop\NLP2SQL\credit products.xlsx")
credit_df

,product_code,product_type
0,10.FOB_IND_0006,Залоговый кредит
1,10.TMB_IND1_0025,Ипотека
2,10.002_0106,НН
3,10.TMB_IND2_0049,Автокредит
4,10.KSNV_IND_CR_0001,Залоговый кредит
...,...,...
257,10.TMB_IND1_0030,Залоговый кредит
258,10.TMB_IND2_0019,Залоговый кредит
259,10.TMB_IND1_0028,Ипотека
260,10.KSNV_IND_CR_0020,Ипотека


In [70]:
zalog_codes=credit_df[credit_df['product_type']=='Залоговый кредит']['product_code'].to_list()
ipoteka_codes=credit_df[credit_df['product_type']=='Ипотека']['product_code'].to_list()
nn_codes=credit_df[credit_df['product_type']=='НН']['product_code'].to_list()
auto_codes=credit_df[credit_df['product_type']=='Автокредит']['product_code'].to_list()
express_codes=credit_df[credit_df['product_type']=='Экспресс кредит']['product_code'].to_list()
tovar_codes=credit_df[credit_df['product_type']=='Товарный кредит']['product_code'].to_list()

In [145]:
text = " Выведи топ 10 тех у кого был наибольший платёж по авто кредиту в марте 2024 года"

final_prompt = f"""
Ты — генератор SQL-запросов по таблице credit_portfolio, которая ежедневно обновляется в ForteBank.

Описание таблицы credit_portfolio:
- Каждая строка отражает статус одного и того же контракта **на определённую дату** (act_date).
- Если контракт открыт, он будет присутствовать **в каждой дате** от open_date до закрытия (close_date или liquidate_date), даже если уже погашен.
- Таким образом, в таблице ежедневно представлены все кредиты: открытые и уже закрытые — с их актуальными статусами.

Столбцы таблицы:
  • act_date (date) — дата записи; каждый контракт появляется в каждой дате от open_date до coalesce(liquidate_date,close_date), даже если уже закрыт  
  • contract_number (varchar) — номер контракта; у одного bin_iin может быть несколько контрактов  
  • bin_iin (varchar) — БИН/ИИН клиента  
  • agreem_state (varchar) — статус (Например, «Погашен», «Актуален», «Отказ Банком» и т.д.)  
  • contract_amount (numeric) —  это изначальная сумма кредита, она одинакова для всех дат одного и того же контракта, поэтому если нужно агрегировать по contract_amount, используй только одну строку на контракт
  • open_date (date) — дата открытия  
  • close_date (date) — дата закрытия 
  • liquidate_date (date) — дата досрочной ликвидации (может быть NULL)  
  • currency (varchar) — валюта  
  • sign_restructing (varchar) — была ли реструктуризация (1/0)  
  • branch_code (varchar) — код филиала  
  • branch_name (varchar) — название филиала  
  • loan_purpose_code (varchar) — код цели кредита  
  • loan_purpose_name (varchar) — название цели (покупка, строительство и т.д.)  
  • collector_company (varchar) — коллекторская компания  
  • product_code (varchar) — код продукта  
  • product_name (varchar) — название продукта  
  • interest_rate (numeric) — ставка  
  • eff_interest_rate (numeric) — эффективная ставка  
  • delay_count_main (integer) — просрочка по телу (дней), 
  • delay_count_percent (integer) — просрочка по процентам (дней)  
  • balance_main_dbt_amt_in_lcl_ccy (numeric) — остаток долга в тенге  
  • balance_int_amt_in_lcl_ccy (numeric) — сумма последнего платежа в тенге  
  • next_payment_date (date) — дата следующего платежа  
  • z_good (varchar) — сегмент клиента: Good_bank, Heritage, Stressful  

product_code для каждого кредитного продукта
  – Экспресс кредиты = {express_codes} 
  – Авто кредиты= {auto_codes}
  – Залоговые кредиты = {zalog_codes}
  – Ипотечные кредиты = {ipoteka_codes}
  – Товарные кредиты = {tovar_codes}
  

Типы agreem_state:(
    "Продан (закрыт)", "Отказ заемщика от займа", "Клиент отказался от кредита",
    "Отказ Банком", "Передан на совершение испол.надписи", "Актуален",
    "Передан в суд", "Списан долг", "Договор завершен",
    "Отсутствует на источнике", "Прощен", "Продан", "Погашен",
    "Отказ от кредита", "Списан", "Погашен по решению суда",
    "Удалён физически", "Реф по ГП", "Погашен ГП", "Завершение",
    "Удален", "Договор введен", "Отказ Заемщиком", "Списаны проценты",
    "Отказ заемщиком", "Зарегистрирован", "Графики рассчитаны", "Введен"
)

ВСЕ branch_name: 
(
    'Филиал АО "ForteBank" в г. Атырау',
    'Филиал АО "ForteBank" в г. Караганда',
    'Филиал АО "ForteBank" в г.Уральск',
    'Филиал АО "ForteBank" в г. Екибастуз',
    'Филиал Акционерного Общества ForteBank в городе Астана',
    'Филиал АО "ForteBank" в г. Шымкент',
    'Филиал АО "ForteBank" в г. Талдыкорган',
    'Филиал АО "ForteBank" в г. Кызылорда',
    'Филиал АО "ForteBank" в г. Каскелен',
    'Филиал АО  "ForteBank" в г. Алматы',
    'Филиал АО "ForteBank" в г. Туркестан',
    'Филиал АО "ForteBank" в г. Усть-Каменогорск',
    'Филиал АО "ForteBank" г. Актау',
    'Филиал АО "ForteBank" в г. Семей',
    'Филиал АО "ForteBank" в г. Павлодар',
    'Филиал АО "ForteBank" в г. Тараз',
    'Акционерное общество "ForteBank"',
    'Филиал АО "ForteBank" в г. Актобе',
    'Филиал АО "ForteBank" в г. Костанай',
    'Филиал АО "ForteBank" в г. Кокшетау',
    'Филиал АО "ForteBank" в г. Петропавловск'
)


Правила генерации SQL:
- Всегда используй act_date для фильтрации по дате (например, `act_date BETWEEN '2025-03-01' AND '2025-03-31'`)
- Учитывай, что одни и те же контракты появляются в таблице ежедневно с их текущим статусом
- Чтобы не дублировать контракты, всегда используй `COUNT(DISTINCT contract_number)` или `DISTINCT` там, где нужно
- Если не говориться а каком-то определённом виде кредита то не фильтруй по  product_code
- **Сделай всё в одном SQL-запросе**
-Если указано "максимальная просрочка N дней", то нужно: взять максимальное из delay_count_main и delay_count_percent
- если нужно вытащить сумму оставшегося долга за какой то период то надо брать минимум 
- в столбцах delay_count_main, delay_count_percent null значит просрочки нет,а просрочка начинается с 0  

Пример: если спросят "Сколько было актуальных кредитов в марте 2025 года", правильный SQL:
```sql
SELECT COUNT(DISTINCT contract_number)
FROM credit_portfolio
WHERE act_date BETWEEN '2025-03-01' AND '2025-03-31'
  AND agreem_state = 'Актуален'

теперь подумай хорошо и ответь на данный вопрос: {text}

"""

final_response = client_api.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are an NLP-to-SQL generator for a daily-updated credit_portfolio table in ForteBank. "},
        {"role": "user", "content": final_prompt}
    ],
    temperature=0
)
final_result = final_response.choices[0].message.content.strip()


print(final_result)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************q-AA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [89]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Параметры генерации
n_transactions = 10000
start_date = pd.to_datetime("2024-01-01")
end_date = pd.to_datetime("2025-05-01")

ps_types = ["VISA", "MS"]
device_titles = [
    "АТМ", "Технологические транзакции", "КАССА", "ПОС-ТЕРМИНАЛ",
    "Телебанк-транзакции", "ИМПРИНТЕР", "FIMI"
]

# Генерация клиентов
n_clients = 3000
client_iins = [str(900000000000 + i) for i in range(n_clients)]
client_fios = [f"Иванов Иван {i}" for i in range(n_clients)]
client_birthdays = [datetime(1980, 1, 1) + timedelta(days=random.randint(0, 15000)) for _ in range(n_clients)]
client_phones = [f"+7701{random.randint(1000000, 9999999)}" for _ in range(n_clients)]
clients = list(zip(client_iins, client_fios, client_birthdays, client_phones))

# Функция генерации одной строки транзакции
def generate_transaction(i):
    transdatetime = start_date + timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds())))
    ps = random.choice(ps_types)
    device_title = random.choice(device_titles)
    amount = round(random.uniform(1000, 500_000), 2)
    is_zp = random.choice([0, 1])
    is_cash_withdrawal = random.choice([0, 1])
    docno = f"DOC{i:07d}"
    is_zp_trn = 1 if is_zp and random.random() < 0.7 else 0
    client = random.choice(clients)

    return {
        "transdate": transdatetime,
        "ps": ps,
        "device_title": device_title,
        "amount": amount,
        "is_zp": is_zp,
        "is_cash_withdrawal": is_cash_withdrawal,
        "client_fio": client[1],
        "client_iin": client[0],
        "docno": docno,
        "client_birthday": client[2].date(),
        "card_main_phone": client[3],
        "is_zp_trn": is_zp_trn
    }

# Генерация всех транзакций
transdoc = pd.DataFrame([generate_transaction(i) for i in range(n_transactions)])

# Проверка
print(transdoc.head())
print(f"{transdoc['client_iin'].nunique()} уникальных клиентов на {n_transactions} транзакций")


            transdate    ps                device_title     amount  is_zp  \
0 2025-03-11 18:41:47  VISA                        FIMI  192780.77      0   
1 2024-01-04 21:15:16  VISA                        FIMI    4524.99      1   
2 2024-03-19 23:13:11    MS  Технологические транзакции  462385.79      1   
3 2024-12-02 02:23:28  VISA         Телебанк-транзакции  390798.13      1   
4 2024-05-20 13:50:07  VISA                         АТМ   56054.61      1   

   is_cash_withdrawal        client_fio    client_iin       docno  \
0                   1   Иванов Иван 741  900000000741  DOC0000000   
1                   1  Иванов Иван 2693  900000002693  DOC0000001   
2                   0  Иванов Иван 1304  900000001304  DOC0000002   
3                   1  Иванов Иван 1874  900000001874  DOC0000003   
4                   1  Иванов Иван 1809  900000001809  DOC0000004   

  client_birthday card_main_phone  is_zp_trn  
0      2015-09-12    +77011845005          0  
1      1998-11-25    +770169

In [90]:
transdoc

,transdate,ps,device_title,amount,is_zp,is_cash_withdrawal,client_fio,client_iin,docno,client_birthday,card_main_phone,is_zp_trn
0,2025-03-11 18:41:47,VISA,FIMI,192780.77,0,1,Иванов Иван 741,900000000741,DOC0000000,2015-09-12,+77011845005,0
1,2024-01-04 21:15:16,VISA,FIMI,4524.99,1,1,Иванов Иван 2693,900000002693,DOC0000001,1998-11-25,+77016957047,0
2,2024-03-19 23:13:11,MS,Технологические транзакции,462385.79,1,0,Иванов Иван 1304,900000001304,DOC0000002,2020-05-29,+77012634537,0
3,2024-12-02 02:23:28,VISA,Телебанк-транзакции,390798.13,1,1,Иванов Иван 1874,900000001874,DOC0000003,1994-08-20,+77014174480,1
4,2024-05-20 13:50:07,VISA,АТМ,56054.61,1,1,Иванов Иван 1809,900000001809,DOC0000004,2010-06-02,+77012057493,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-07-28 16:27:56,MS,ПОС-ТЕРМИНАЛ,484137.97,1,0,Иванов Иван 2854,900000002854,DOC0009995,2011-09-04,+77015452521,0
9996,2024-10-18 17:08:31,MS,КАССА,183431.93,0,1,Иванов Иван 765,900000000765,DOC0009996,1994-07-11,+77018689938,0
9997,2024-11-25 02:47:42,VISA,ПОС-ТЕРМИНАЛ,281850.34,1,1,Иванов Иван 1708,900000001708,DOC0009997,1987-08-23,+77015693134,0
9998,2024-06-01 03:18:51,VISA,КАССА,25939.54,1,0,Иванов Иван 492,900000000492,DOC0009998,1989-10-22,+77012006284,0
